# DCM - Manual Report Refresh

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from brand_reporting_dags.dfa_reporting_dag_tasks import dfa_report_extract,dfa_report_load,clean_up

import json
import os
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore", "Your application has authenticated using end user credentials")

In [4]:
from rtf_utils.dfa_utils import CampaignManagerReport,get_dfa_report,clean_dcm_file
from rtf_utils.gcp_utils import BigQuery,CloudStorage
from essence.analytics.platform import securedcredentials as secure_creds

## DCM Reports

In [3]:
with open('dags/brand_report_task_queue.json') as json_file:
    config_json = json.load(json_file)
    campaigns = config_json['campaigns']

Task = (report_id, bq_dest (proj:dataset.table), schema)

In [4]:
tasks = campaigns[0]['dcmTasks']

In [34]:
tasks = [
{'name':'classification','profileId':5096586,'reportId':641053372},
{'name':'classification','profileId':5096586,'reportId':641055447},
{'name':'classification','profileId':5096586,'reportId':641053269},
{'name':'geo_report','profileId':5096586,'reportId':641050621},
{'name':'geo_report','profileId':5096586,'reportId':641055789},
{'name':'geo_report','profileId':5096586,'reportId':641046828},
{'name':'video_player_size','profileId':5096586,'reportId':641052858},
{'name':'video_player_size','profileId':5096586,'reportId':641055441},
{'name':'video_player_size','profileId':5096586,'reportId':641053414}]

In [5]:
service_account_email='131786951123-compute@developer.gserviceaccount.com'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'essence-analytics-dwh'
credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)

In [36]:
## Update and run report
start_date = '2019-10-01'
end_date = '2019-10-20'

check_q = []

for task in tasks:
    
    report = CampaignManagerReport(credentialsFromVault,profile_id = 5096586,report_id = task['reportId'])
    print("Fetch report meta")
    report.get_report()
    
    print("set date")
    report.set_date_range(start_date,end_date)
    print("run_report")
    
    report.run_report()
    
    check_q.append(report)

Fetch report meta
set date
run_report
Fetch report meta
set date
run_report
Fetch report meta
set date
run_report
Fetch report meta
set date
run_report
Fetch report meta
set date
run_report
Fetch report meta
set date
run_report
Fetch report meta
set date
run_report
Fetch report meta
set date
run_report
Fetch report meta
set date
run_report


In [42]:
import time
for report in check_q:
    #time.sleep(10)
    print(report.get_report_status())

REPORT_AVAILABLE
REPORT_AVAILABLE
REPORT_AVAILABLE
REPORT_AVAILABLE
REPORT_AVAILABLE
REPORT_AVAILABLE
REPORT_AVAILABLE
REPORT_AVAILABLE
REPORT_AVAILABLE


In [51]:
import time,csv
files = []
for report in check_q:
    #time.sleep(10)
    try:
        filename = report.download_file()
        files.append(filename)
        print("Saved @ {}".format(filename))
    except:
        print("bad news hombre")

    clean_dcm_file(filename)

gcs = CloudStorage(credentialsFromVault)       
for file in files:    
    blob_name = "brand_reporting/" + file
    gcs.upload_blob('rtf_staging', blob_name, file, mode="filename")

Saved @ Q4_BR_Reporting_Classification_Group_2.csv
Saved @ Q4_BR_Reporting_Geo_Report_Group_3.csv
Saved @ Q4_BR_Reporting_Classification_Group_1.csv
Saved @ Q4_BR_Reporting_Geo_Report_Group_2.csv
Saved @ Q4_BR_Reporting_Classification_Group_3.csv
Saved @ Q4_BR_Reporting_Geo_Report_group1.csv
Saved @ Q4_BR_Reporting_Video_Player_Size_Group_2.csv
Saved @ Q4_BR_Reporting_Video_Player_Size_Group_3.csv
Saved @ Q4_BR_Reporting_Video_Player_Size_Group_1.csv


In [11]:
service_account_email='131786951123-compute@developer.gserviceaccount.com'
credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)
bq = BigQuery(credentialsFromVault)

for file in files:

    bucket = 'rtf_staging/brand_reporting'
    file_uri = "gs://" + bucket + "/" + filename
        
    dataset_id = 'RTF_DWH_CampaignManager'
    
    if 'Classification' in file_uri:
        dest_table = 'DCM_contentReport'
    elif 'Geo' in file_uri:
        dest_table = 'DCM_geoReport' 
    else:
        dest_table = 'DCM_playerSizeReport'
    print("Loading {} into {}".format(file_uri,dest_table))
    bq.load_from_gcs(dataset_id,file_uri,dest_table,mode='Append')

Loading gs://rtf_staging/5295_BR_Reporting_Classification_20190913_142345_2656806431.csv into DCM_contentReport
Loading gs://rtf_staging/5295_BR_Reporting_Geo_Report_20190913_144251_2656783687.csv into DCM_geoReport
Loading gs://rtf_staging/5295_BR_Reporting_Video_Player_Size_20190913_140102_2656781516.csv into DCM_playerSizeReport


In [47]:
def clean_dcm_file(filename):
    data = []
    write = False
    with open(filename,'r') as f:
        reader = csv.reader(f, delimiter=',')

        for row in reader:
            if write == True:
                data.append(row)        
            elif row == ['Report Fields']:
                write = True

        if data[-1][0] == 'Grand Total:':
            data.pop()

    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(data)

In [48]:
for file in files:
    clean_dcm_file(file)

FileNotFoundError: [Errno 2] No such file or directory: 'Q4_BR_Reporting_Classification_Group_2.csv'

In [56]:
bq = BigQuery(credentialsFromVault)
bq.load_from_gcs('RTF_DWH_CampaignManager',"gs://rtf_staging/brand_reporting/Q4_BR_Reporting_Classification*","DCM_Classification",extension='csv',mode='Append')

In [57]:
help(bq.load_from_gcs)

Help on method load_from_gcs in module rtf_utils.gcp_utils:

load_from_gcs(dataset_id, file_uri, dest_table, schema=None, mode=None, extension='csv', **kwargs) method of rtf_utils.gcp_utils.BigQuery instance
    Gets data for tile dimenions/filters within data range. 
    
    Cleans and saves file locally
    
    Args:
        dataset_id (int): request start date in YYYYMMDD
        file_uri (str): request end date in YYYYMMDD        
        
        dest_table (str): API token
    
        schema (list): list of BigQuery.SchemaObject. defaults to none which inferes schemea
        
        mode (str)(opt): defaults to WRITE_TRUNCATE disposition
        
        extension: (str): csv or json
    
    Returns:
        filename (str): filename of local file in working dir



In [11]:
bq = BigQuery(credentialsFromVault)
bq.load_from_gcs('RTF_DWH_CampaignManager',"gs://rtf_staging/brand_reporting/Q4_BR_Reporting_Video_Player_Size_Group_1.csv","003_DCM_PlayerSize",extension='csv')
bq.load_from_gcs('RTF_DWH_CampaignManager',"gs://rtf_staging/brand_reporting/Q4_BR_Reporting_Geo_Report_group1.csv","003_DCM_Geo",extension='csv')
bq.load_from_gcs('RTF_DWH_CampaignManager',"gs://rtf_staging/brand_reporting/Q4_BR_Reporting_Classification_Group_1.csv","003_DCM_Classification",extension='csv')

In [7]:
geo = CampaignManagerReport(credentialsFromVault,profile_id = 5096586,report_id =645949948)

In [8]:
geo.__dict__

{'profile_id': 5096586,
 'report_id': 645949948,
 'service': <googleapiclient.discovery.Resource at 0x7f15f4301e48>}